In [ ]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='50c32119-5dc2-4f0f-a296-969f29e3c6f0', project_access_token='p-f316b366e9833f57c9ceef2c4e03405ca7505f82')
pc = project.project_context


# Coursera capstone project notebook

# Segmenting and Clustering Neighborhoods in Toronto

# Web scrapping the data 

In [123]:
import pandas as pd
import numpy as np
import requests
import lxml.html as lh
import pandas as pd

### Scrape Table Cells

In [124]:

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements =doc.xpath('//tr')

### Check the length of the first 12 rows is same

In [125]:

[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

### Parse Table Header

In [135]:
#Parse table header

tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:%s'%(i,name))
    col.append((name,[]))

1:Postcode
2:Borough
3:Neighbourhood



### CREATE THE PANDAS DATAFRAME

In [136]:



#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1       

### ensure that all the columns have the same length.

In [143]:
[len(C) for (title,C) in col]

[289, 289, 289]

### Create data frame

In [182]:
Dict={title:column for (title,column) in col}
df1=pd.DataFrame(Dict)

In [183]:
df1.columns

Index(['Borough', 'Neighbourhood\n', 'Postcode'], dtype='object')

In [186]:
df1.head()


,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned\n,M1A
1,Not assigned,Not assigned\n,M2A
2,North York,Parkwoods\n,M3A
3,North York,Victoria Village\n,M4A
4,Downtown Toronto,Harbourfront\n,M5A


In [187]:
df1['Borough'].value_counts()

Not assigned        77
Etobicoke           45
Scarborough         38
North York          38
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

### Remove the 'Not assigned' Borough

In [188]:
df2 = df1[df1.Borough != 'Not assigned']
df2.head()

,Borough,Neighbourhood,Postcode
2,North York,Parkwoods\n,M3A
3,North York,Victoria Village\n,M4A
4,Downtown Toronto,Harbourfront\n,M5A
5,Downtown Toronto,Regent Park\n,M5A
6,North York,Lawrence Heights\n,M6A


### Inspect the not assigned neighbourhood

In [201]:
dfN = df2[df2.Neighbourhood == 'Not assigned\n']
dfN.head()

AttributeError: 'DataFrame' object has no attribute 'Neighbourhood'

### Merge the duplicated Postcode and Borough Neighborhoods

In [193]:
df3 = df2.groupby(['Borough','Postcode'])['Neighbourhood\n'].apply(','.join).reset_index()
df.head()

,Borough,Postcode,Neighbourhood
0,Central Toronto,M4N,Lawrence Park\n
1,Central Toronto,M4P,Davisville North\n
2,Central Toronto,M4R,North Toronto West\n
3,Central Toronto,M4S,Davisville\n
4,Central Toronto,M4T,"Moore Park\n,Summerhill East\n"


In [ ]:
#if a cell has a borough and not assigned a neighbourhood the borough becomes the neighbourhood.
#loop thru neighbourhood if 'not assigned' is there replace with borough value in same index
df2.loc[:,'Neighbourhood'] = df['Neighbourhood'].where((df['Neighbourhood']='Not assigned'))
